# TRANSFER LEARNING

**_Experimenting with Transfer Learning for a classification task._**

In transfer learning, features learned on one problem are taken and leveraged them on a new and similar problem. It is usually done for tasks where dataset has too little to train a full-scale model from scratch. 

**The Experiment:**

- Loads _MobileNetV2_ model - pretrained on _imagenet_ dataset, as a base model taking all layers except the top one that is used for classification specific to ImageNet task.

- Freezes all the layers in the base model to avoid destroying already learned parameters during training related to new task to classify cats and dogs.

- Adds few new, trainable layers such as pooling, dropout and dense layer on top of the frozen layers for them to learn based on the new dataset.

- Prepares the new dataset in a form that is acceptable by the base model.

- Trains the head (newly added layers) on the new data over few epochs to get them trained.

- Unfreezes the all the layers in the base model and fine-tunes the entire model by re-training it on the new data with a very low learning rate to achieve meaningful improvements by incrementally adapting the pretrained features to the new data.

- Evaluates the model performance on the test data.

## Importing Packages

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import data as tf_data

import matplotlib.pyplot as plt

## Data Acquisition

In [ ]:
# Fetches the "cats vs. dogs" dataset using TFDS
# Only 40% of the data is used to show the effectiveness of transfer learning
train_set, val_set, test_set = tfds.load(
    "cats_vs_dogs",
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],  # 40% for training, 10% for validation and 10% for test
    as_supervised=True,                                         # Includes labels
)

print("Sample Counts:")
print(f"Training: {train_set.cardinality()}, Validation: {val_set.cardinality()}, Test: {test_set.cardinality()}")

## Data Analysis

In [ ]:
# Plots few of the samples from the train set

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 8))
axes = axes.flatten()                                       # Flattens 2D array of axes to 1D for easier iteration
for idx, (image, label) in enumerate(train_set.take(8)):    # Iterates through sample images to plot them
    axes[idx].imshow(image)                                 # Displays image in corresponding subplot
    axes[idx].set_title(int(label))                         # Sets title of subplot to image label
    axes[idx].axis("off")                                   # Hides axis for a cleaner look

plt.suptitle("Samples from Cats. vs. Dogs Training set", fontsize=16)   # Sets figure title
plt.tight_layout()                                          # Adjusts layout to prevent overlap
plt.show()

## Data Preprocessing

As the images size varies and each pixel consists of 3 integer values between 0 and 255 (RGB level values), all the images get resized to same size of 160x160 and pixel values get normalized between -1 and 1 according to the requirement of the pretrained model. Additionally, random data augmentation is also applied.

**Resizing images**

In [ ]:
resizer = # Code here to use `tf.keras.layers.Resizing` as function to resize image to 160x160

# Maps each sample from the each of the datasets and performs resizing
# Perform the image resizing by calling `map` lambda function of each dataset 
# passing resizer (with labels)

train_set = # Code here
val_set = # Code here
test_set = # Code here

In [ ]:
# Confirms the resizing just by taking one sample with an index
next(iter(train_set))[0].shape

**Augmenting Data**

Applying data augmentation to generate samples artificially by applying random and realistic transformations to the training images, such as random horizontal flipping or small random rotations helping the model to get exposed to different aspects of the training data while slowing down overfitting. It makes sense when dataset is not large or not balanced.

In [ ]:
# Configures the sequence of operations (once again taking layers as utulity functions)
augmentation_layers = [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
]

def data_augmentation(x):
    """
    Augments the image by flipping and rotating it
    """
    for layer in augmentation_layers:
        x = layer(x)
    return x

# Use `data_augmentation` function passing it to train set's `map` lambda function to augment the images
train_set = # Code here


**Batching Data and Optimizing Loading Speed using Prefetching and Caching**

In [7]:
batch_size = 32

train_set = train_set.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
val_set = val_set.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test_set = test_set.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

## Modeling

In [ ]:
# Instantiate `tf.keras.applications.MobileNetV2` as base model specifying 
# (160, 160, 3) as `input_shape`, 'False` to `include_top` to not to include top layer and
# `imagenet` as `weights` to initialize the model with specific weights

base_model = tf.keras.applications.MobileNetV2(
    # Code here
    ...
    ...
)

# Code to freeze all layers in the base model

In [ ]:
# Create target model out of base model

inputs = # Code to initialize the model input with expectation of `shape` (160, 160, 3)

x = # Code to pass `inputs` to routine `tf.keras.applications.mobilenet_v2.preprocess_input` to scale input pixels between -1 and 1


x = # Code to pass `x` to base model `base_model`. Also set `training` parameter to `False` to keep model in inference mode

x = # Code to call `tf.keras.layers.GlobalAveragePooling2D()` passing `x` to convert base model's multi-dimensional output (`base_model.output_shape[1:]`) to vectors

x = # Code to call `tf.keras.layers.Dropout` passing argument 0.2 to initialize and then pass `x` to initialized class to regularize the network with 20% dropout

outputs = # Call to `tf.keras.layers.Dense` with initialize a dense layer with 1 unit and then pass `x` to the layer

model = # Create target model calling `tf.keras.Model` passing both `inputs` and `outputs` as arguments

In [ ]:
model.summary(show_trainable=True)      # Shows the model summary with both trainable and non-trainable parameters

## Training the Model

### Training Head of the Model

In [ ]:
# Compile the model by calling model's `compile` method passing `adam` as `optimizer`, `tf.keras.losses.BinaryCrossentropy(from_logits=True)` as `loss`,
# [tf.keras.metrics.BinaryAccuracy()] as metrics
# Code here

# Call model's `fit` method to fit the model on train set with 2 epochs. 
# Also pass validation set to parameter `validation_data` to measures the learning during training
# Code here

In [ ]:
# Evaluates the model performance post head training

post_head_train_perf = model.evaluate(test_set)

print(f"Test set performance after head training:\n \
      Loss: {post_head_train_perf[0]:.2f}, Accuracy: {post_head_train_perf[1] * 100:.2f}%")

### Fine-tuning the Model
Unfreezing the base model and training the entire model end-to-end with a low learning rate.

In [ ]:
# Code to set (all layers of) base model as not-trainable

In [ ]:
model.summary(show_trainable=True)

In [ ]:
# Compile the model by calling model's `compile` method passing `tf.keras.optimizers.Adam(1e-5)` as `optimizer`, 
# `tf.keras.losses.BinaryCrossentropy(from_logits=True)` as `loss`,
# [tf.keras.metrics.BinaryAccuracy()] as metrics
# Code here

# Call model's `fit` method to fit the model on train set with 5 epochs. 
# Also pass validation set to parameter `validation_data` to measures the learning during training
# Code here

In [ ]:
# Evaluates the model performance post fine-tuning the full model

post_full_model_train_perf = model.evaluate(test_set)

print(f"Test set performance after full model training:\n \
      Loss: {post_full_model_train_perf[0]:.2f}, Accuracy: {post_full_model_train_perf[1] * 100:.2f}%")

**Observations:**

- Did the model learn over less data? Explain in detail.

- Why is the reason data augmentation was used for?

- List general workflow of transfer learning followed in this experiment.